<img src="../../img/logo_amds.png" alt="Logo" style="width: 128px;"/>

# AmsterdamUMCdb - Freely Accessible ICU Database

version 1.0.2 March 2020  
Copyright &copy; 2003-2020 Amsterdam UMC - Amsterdam Medical Data Science

# Mechanical ventilation
Shows the modes or types of mechanical ventilation (artificial respiration), if any, a patient received. Includes common ventilation parameters: ventilation mode, peak airway pressure, PEEP, respiratory rate, tidal volume, minute volume/ventilation, end tidal CO2.

**Note**: Requires creating the [dictionaries](../../dictionaries/create_dictionaries.ipynb) before running this notebook.

## Imports

In [1]:
%matplotlib inline
import amsterdamumcdb
import psycopg2
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl

import io
from IPython.display import display, HTML, Markdown

## Display settings

In [2]:
#matplotlib settings for image size
#needs to be in a different cell from %matplotlib inline
plt.style.use('seaborn-darkgrid')
plt.rcParams["figure.dpi"] = 288
plt.rcParams["figure.figsize"] = [16, 12]
plt.rcParams["font.size"] = 12

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000

## Connection settings

In [3]:
#Modify config.ini in the root folder of the repository to change the settings to connect to your postgreSQL database
import configparser
import os
config = configparser.ConfigParser()

if os.path.isfile('../../config.ini'):
    config.read('../../config.ini')
else:
    config.read('../../config.SAMPLE.ini')

#Open a connection to the postgres database:
con = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'])
con.set_client_encoding('WIN1252') #Uses code page for Dutch accented characters.
con.set_session(autocommit=True)

cursor = con.cursor()
cursor.execute('SET SCHEMA \'amsterdamumcdb\''); #set search_path to amsterdamumcdb schema

In [4]:
#loads the dictionary for fast lookup
listitems_itemids = pd.read_csv('../../dictionaries/listitems_itemids.csv')
numericitems_itemids = pd.read_csv('../../dictionaries/numericitems_itemids.csv')

## Common ventilation parameters

### ventilation mode

In [5]:
listitems_itemids[listitems_itemids['item'].str.contains('(?<!medicatie.)(?<!SEPSIS_PROTECTIEVE_)(?<!Acceptatie.)(?:beademing|ventilatie|toediening|bipap)', flags=re.IGNORECASE)]

,itemid,item,valueid,value
36,6685,Type Beademing Evita 4,1,CPPV
37,6685,Type Beademing Evita 4,3,ASB
38,6685,Type Beademing Evita 4,5,CPPV/ASSIST
39,6685,Type Beademing Evita 4,6,SIMV/ASB
40,6685,Type Beademing Evita 4,8,IPPV
41,6685,Type Beademing Evita 4,9,IPPV/ASSIST
42,6685,Type Beademing Evita 4,10,CPAP
43,6685,Type Beademing Evita 4,11,CPAP/ASB
44,6685,Type Beademing Evita 4,12,MMV
45,6685,Type Beademing Evita 4,13,MMV/ASB


### Peak airway pressure

In [6]:
numericitems_itemids[numericitems_itemids['item'].str.contains('(?<!PA.)(?<!PBE.|PD1.|PFD.|TMP.|Edi.)(?<!Flow.)(?:peak|piek|max)', 
                                                               flags=re.IGNORECASE)]

,itemid,item,unitid,unit
202,8852,P max,177,mbar
222,8877,Peak druk,177,mbar
881,12281,Piek druk,175,cmH2O
926,12365,Piek druk (2),175,cmH2O
1211,16239,Zephyros Ppeak,175,cmH2O


### PEEP

In [7]:
numericitems_itemids[numericitems_itemids['item'].str.contains('(?<!T.)(?<!boven.)(?<!Intrinsic.)PEEP(?!.tot)', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
211,8862,PEEP/CPAP,177,mbar
224,8879,PEEP (gemeten),177,mbar
283,9666,PEEP (gemeten)(2),177,mbar
884,12284,PEEP (Set),175,cmH2O
911,12336,PEEP (Set) (2),175,cmH2O
1220,16250,Zephyros PEEP,175,cmH2O


### respiratory rate

In [8]:
numericitems_itemids[numericitems_itemids['item'].str.contains(r'(?<!hfo-)(?<!hart)(?<!pacemaker )freq', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
126,7726,Ademfrequentie Spontaan,0,None
200,8850,Frequentie,0,None
203,8853,Freq. IMV,0,None
218,8873,Ademfrequentie Evita,0,None
219,8874,Ademfrequentie Monitor,0,None
221,8876,Frequentie Spontaan,0,None
277,9654,Ademfrequentie Spontaan(2),0,None
871,12266,Ademfreq.,15,/min
883,12283,Adem Frequentie (Set),15,/min
887,12287,SIMV Frequentie (Set),26,l/min


### tidal volume

In [9]:
numericitems_itemids[numericitems_itemids['item'].str.contains(r'(?<!CO2.)(?:teug|tidal)(?!.CO2)', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
201,8851,Tidal Volume (Set),6,ml
217,8872,Tidal Volume Spirometer,6,ml
276,9646,Tidal Volume (set)(2),6,ml
875,12275,Exp. tidal volume,6,ml
877,12277,Insp. tidal volume,6,ml
890,12291,Teugvolume (Set),6,ml
922,12358,Exp. tidal volume (2),6,ml
923,12360,Insp. tidal volume (2),6,ml


### minute volume (=minute ventilation)

In [10]:
numericitems_itemids[numericitems_itemids['item'].str.contains(r'(?<!tidal.)(?<!teug)(?:min.*vol|min.*vent|vol.*set)',
                                                               flags=re.IGNORECASE)]

,itemid,item,unitid,unit
874,12273,Exp. minuut volume,26,l/min
876,12276,Insp. Minuut volume,26,l/min
889,12289,Volume (Set),26,l/min
921,12357,Exp. minuut volume (2),26,l/min
932,12374,MinVent,26,l/min


### end tidal CO2

In [11]:
numericitems_itemids[numericitems_itemids['item'].str.contains(r'(?:tidal|Et).*CO2', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
32,6707,End tidal CO2 concentratie,173,mmHg
228,8884,End Tidal CO2 %,0,None
229,8885,End Tidal CO2 mmHG,173,mmHg
279,9658,End Tidal CO2% (2),0,None
920,12356,End tidal CO2 concentratie (2),173,mmHg
1018,12805,EtCO2 (%),191,Geen


### Mechanical ventilation: summary per admission

In [12]:
sql_mechanical_ventilation = """
SELECT 
    admissionid,
    CASE
        WHEN COUNT(*) > 0 THEN TRUE
        ELSE FALSE
    END AS mechanical_ventilation_bool,
    STRING_AGG(DISTINCT value, '; ') AS mechanical_ventilation_modes
FROM listitems
WHERE 
    (
        itemid = 9534  --Type beademing Evita 1
        AND valueid IN (
            1, --IPPV
            2, --IPPV_Assist
            3, --CPPV
            4, --CPPV_Assist
            5, --SIMV
            6, --SIMV_ASB
            7, --ASB
            8, --CPAP
            9, --CPAP_ASB
            10, --MMV
            11, --MMV_ASB
            12, --BIPAP
            13 --Pressure Controled
        )
    )
    OR (
        itemid = 6685 --Type Beademing Evita 4
        AND valueid IN (
            1, --CPPV
            3, --ASB
            5, --CPPV/ASSIST
            6, --SIMV/ASB
            8, --IPPV
            9, --IPPV/ASSIST
            10, --CPAP
            11, --CPAP/ASB
            12, --MMV
            13, --MMV/ASB
            14, --BIPAP
            20, --BIPAP-SIMV/ASB
            22 --BIPAP/ASB
        )
    )
    OR (
        itemid = 8189 --Toedieningsweg O2
        AND valueid = 16 --CPAP
    ) 
    OR (
        itemid IN (
            12290, --Ventilatie Mode (Set) - Servo-I and Servo-U ventilators
            12347 --Ventilatie Mode (Set) (2) Servo-I and Servo-U ventilators
        )
        AND valueid IN (
            --IGNORE: 1, --Stand By
             2, --PC
             3, --VC
             4, --PRVC
             5, --VS
             6, --SIMV(VC)+PS
             7, --SIMV(PC)+PS
             8, --PS/CPAP
             9, --Bi Vente
             10, --PC (No trig)
             11, --VC (No trig)
             12, --PRVC (No trig)
             13, --PS/CPAP (trig)
             14, --VC (trig)
             15, --PRVC (trig)
             16, --PC in NIV
             17, --PS/CPAP in NIV
             18 --NAVA
        )
    )
    OR itemid = 12376 --Mode (Bipap Vision)
    AND valueid IN (
        1, --CPAP
        2 --BIPAP
    )
GROUP BY admissionid
"""            
mechanical_ventilation = pd.read_sql(sql_mechanical_ventilation,con)
mechanical_ventilation.head()

,admissionid,mechanical_ventilation_bool,mechanical_ventilation_modes
0,0,True,PC ; PS/CPAP; PS/CPAP (trig)
1,1,True,PC ; PS/CPAP; PS/CPAP (trig)
2,2,True,PC
3,3,True,PC (No trig); PS/CPAP
4,4,True,PC ; PC (No trig); PS/CPAP; PS/CPAP (trig); VC
